In [22]:
from client import Client
from global_model import Global_model
from sklearn.utils import shuffle
from supported_modles import Supported_modles

import random

In [23]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [24]:
selected_model = Supported_modles.MLP_classifier

In [25]:
#NODEs
client1 = Client("node1","0.0.0.0")
dataset = client1.load_data("../../datasets/UNSW_NB15_training-set.csv")
client2 = Client("node2","0.0.0.0")
client3 = Client("node3","0.0.0.0")
client4 = Client("node4","0.0.0.0")

dataset = shuffle(dataset)

clients = [client1, client2, client3, client4 ]

client1.preprocess_data(dataset)

X = client1.x
y = client1.y

client1.x = X[:20000]
client1.y = y[:20000]

client2.x = X[20000:40000]
client2.y = y[20000:40000]

client3.x = X[40000:50000]
client3.y = y[40000:50000]

client4.x = X[60000:]
client4.y = y[60000:]

In [26]:
f1_before = []
for client in clients:
    print(client.y.sum())
    client.split_data()
    client.train_model(selected_model)
    print(f'Model accuracy: {client.accuracy}')
    print(f'Model f1 score: {client.F1}')
    f1_before.append(client.F1)

11014
Model accuracy: 0.9015151515151515
Model f1 score: 0.9115646258503401
11066
Model accuracy: 0.9078787878787878
Model f1 score: 0.9203771608171818
5397
Model accuracy: 0.9157575757575758
Model f1 score: 0.9240851993446205
12280
Model accuracy: 0.9202170963364993
Model f1 score: 0.9289855072463767


In [27]:
global_model = Global_model(selected_model)
global_model.init_global_model(client1.x.shape[1], selected_model)

In [28]:
number_of_rounds = 5
for _ in range(number_of_rounds):
    print(f'Starting new round!')
    f1s = []
    weigths = []

    applicable_clients = random.sample((clients), k=3)

    for client in applicable_clients:
        client.test_model_f1()
        f1s.append(client.F1)
    
    for x in range(len(applicable_clients)):
        w = f1s[x]/(sum(f1s))
        weigths.append(w)
    print(f'Weights: {weigths}')

    global_model.update_global_model(applicable_clients,weigths,selected_model)
    
    for client in clients:
        f1_local = client.test_model_f1()
        if selected_model == Supported_modles.MLP_classifier:
            int_local = client.model.intercepts_
            coef_local = client.model.coefs_
            client.model.intercepts_ = global_model.model.intercepts_
            client.model.coefs_ = global_model.model.coefs_
        else:
            int_local = client.model.intercept_
            coef_local = client.model.coef_
            client.model.intercept_ = global_model.model.intercept_
            client.model.coef_ = global_model.model.coef_
        print("f1 local: ", f1_local)

        f1_fede = client.test_model_f1()
        print("f1 fede: ", f1_fede)
        if f1_fede < f1_local:
            # local was better set to local values
            if selected_model == Supported_modles.MLP_classifier:
                client.model.intercepts_ = int_local
                client.model.coefs_ = coef_local
            else:
                client.model.intercept_ = int_local
                client.model.coef_ = coef_local
            print("f1 local was better")
        elif f1_fede == f1_local:
            print("the same")
        else:
            print("f1 fede was better")
            client.F1 = f1_fede
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Weights: [0.32972327856541367, 0.3342521122048304, 0.3360246092297559]
f1 local:  0.9115646258503401
f1 fede:  0.8035153797865662
f1 local was better
f1 local:  0.9203771608171818
f1 fede:  0.799755426475084
f1 local was better
f1 local:  0.9240851993446205
f1 fede:  0.8032581453634084
f1 local was better
f1 local:  0.9289855072463767


/home/tk/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:509: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


f1 fede:  0.8120155038759689
f1 local was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Weights: [0.33016611044403976, 0.33647590405375877, 0.33335798550220147]
f1 local:  0.9115646258503401
f1 fede:  0.7588527479570581
f1 local was better
f1 local:  0.9203771608171818
f1 fede:  0.7573300062383032
f1 local was better
f1 local:  0.9240851993446205
f1 fede:  0.7611749680715197
f1 local was better


/home/tk/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:509: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)
/home/tk/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:509: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


f1 local:  0.9289855072463767
f1 fede:  0.7649391107335032
f1 local was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Weights: [0.3318530597358174, 0.33319003764371796, 0.3349569026204646]
f1 local:  0.9115646258503401
f1 fede:  0.759823903473015
f1 local was better
f1 local:  0.9203771608171818
f1 fede:  0.7570657351540171
f1 local was better
f1 local:  0.9240851993446205
f1 fede:  0.7638256343526351
f1 local was better
f1 local:  0.9289855072463767
f1 fede:  0.768231046931408
f1 local was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Weights: [0.32972327856541367, 0.3342521122048304, 0.3360246092297559]
f1 local:  0.9115646258503401
f1 fede:  0.8035153797865662
f1 local was better
f1 local:  0.9203771608171818
f1 fede:  0.799755426475084
f1 local was better
f1 local:  0.9240851993446205
f1 fede:  0.8032581453634084
f1 local was better
f1 local:  0.9289855072463767
f

/home/tk/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:509: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)
/home/tk/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:509: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


f1 local:  0.9115646258503401
f1 fede:  0.8218754827746023
f1 local was better
f1 local:  0.9203771608171818
f1 fede:  0.8192481203007519
f1 local was better
f1 local:  0.9240851993446205
f1 fede:  0.8160990712074303
f1 local was better
f1 local:  0.9289855072463767
f1 fede:  0.8293347873500546
f1 local was better
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [29]:
i = 0
for client in clients:
    f1_before[i]
    print(f'Difference {client.F1-f1_before[i]}')
    i = i + 1

Difference 0.0
Difference 0.0
Difference 0.0
Difference 0.0
